In [ ]:
# import libraries
%matplotlib inline
import os
import pathlib
import salvus.namespace as sn
from salvus.flow import simple_config as sc
import salvus.mesh.unstructured_mesh as um

import more_itertools
import salvus.mesh.layered_meshing as lm
import salvus.mesh.layered_meshing.meshing_protocol as pm

In [ ]:

# remote site to run the simulation on.
SALVUS_FLOW_SITE_NAME = os.environ.get("SITE_NAME", "local")
PROJECT_DIR = "new250_fault"

In [ ]:
# Creating the domain
d = sn.domain.dim3.UtmDomain.from_spherical_chunk(
    min_latitude=-5.0,
    max_latitude=-7.0,
    min_longitude=141.0,
    max_longitude=144.0,
)

# Have a look at the domain to make sure it is correct.
d.plot()

In [ ]:
# Where to download topography data to.
topo_filename = "gmrt_topography.nc"

# Query data from the GMRT web service.
if not os.path.exists(topo_filename):
    d.download_topography_and_bathymetry(
        filename=topo_filename,
        # The buffer is useful for example when adding sponge layers
        # for absorbing boundaries so we recommend to always use it.
        buffer_in_degrees=0.1,
        resolution="default",
    )

In [ ]:
# Data is loaded to Salvus compatible SurfaceTopography object. It will resample
# to a regular grid and convert to the UTM coordinates of the domain.
# This will later be added to the Salvus project.


t1 = sn.topography.cartesian.SurfaceTopography.from_gmrt_file(
    name="png1_topo",
    data=topo_filename,
    resample_topo_nx=200,
    # If the coordinate conversion is very slow, consider decimating.
    decimate_topo_factor=5,
    # Smooth if necessary.
    gaussian_std_in_meters=0.0,
    # Make sure to pass the correct UTM zone.
    utm=d.utm,
)
t2 = sn.topography.cartesian.SurfaceTopography.from_gmrt_file(
    name="small_avg1",
    data=topo_filename,
    resample_topo_nx=500,
    # If the coordinate conversion is very slow, consider decimating.
    decimate_topo_factor=5,
    # Smooth if necessary.
    gaussian_std_in_meters=10000.0,
    # Make sure to pass the correct UTM zone.
    utm=d.utm,
)

In [ ]:
# visualize topography
t2.ds.dem.T.plot(aspect=1, size=7)
t1.ds.dem.T.plot(aspect=1, size=7)

In [ ]:
_dem = (t1.ds.dem - t1.ds.dem.max()).assign_attrs(
    {"ref": float(t1.ds.dem.max())}
)
dem = _dem.copy(data=_dem)
dem
_dem1 = (t2.ds.dem - t2.ds.dem.max()).assign_attrs(
    {"ref": float(t2.ds.dem.max())}
)
dem1 = _dem1.copy(data=_dem1)

@emerald: below here i've translated your BM file into Salvus' layered meshing format. Soon you will be able to create a layered model directly from a BM file as well. Notice how the models and interfaces I am passing are in a top-down order.

For the "interface topo" below, notice how I am specifying the "thin" near-surface layers with reference to the surface topography itself. This was one of the problems before: The peak-to-peak amplitude of the topography was larger than the layer thickness, causing layers to cross and producing indefinite jacobians.

In [ ]:
materials = [
    # lm.material.elastic.Velocity.from_params(vp=3800.0, vs=2400.0, rho=2360.0),
    lm.material.elastic.Velocity.from_params(vp=3800.0, vs=2400.0, rho=2360.0),
    lm.material.elastic.Velocity.from_params(vp=5400.0, vs=3400.0, rho=2720.0),
    lm.material.elastic.Velocity.from_params(vp=6000.0, vs=3600.0, rho=2740.0),
    lm.material.elastic.Velocity.from_params(vp=6200.0, vs=3800.0, rho=2760.0),
    lm.material.elastic.Velocity.from_params(vp=8000.0, vs=4400.0, rho=3290.0),
]



interfaces_flat = [
    lm.interface.Surface(dem1),
    # lm.interface.Surface(dem.assign_attrs({"ref": dem.ref - 1000.0})),
    lm.interface.Surface(dem1.assign_attrs({"ref": dem1.ref - 3000.0})),
    lm.interface.Hyperplane.at(dem.ref - 10_000.0),
    lm.interface.Hyperplane.at(dem.ref - 20_000.0),
    lm.interface.Hyperplane.at(dem.ref - 45_000.0),
    lm.interface.Hyperplane.at(dem.ref - 60_000.0),
]

interfaces_topo = [
    lm.interface.Surface(dem),
    # lm.interface.Surface(dem.assign_attrs({"ref": dem.ref - 1000.0})),
    lm.interface.Surface(dem.assign_attrs({"ref": dem.ref - 3000.0})),
    lm.interface.Hyperplane.at(dem.ref - 10_000.0),
    lm.interface.Hyperplane.at(dem.ref - 20_000.0),
    lm.interface.Hyperplane.at(dem.ref - 45_000.0),
    lm.interface.Hyperplane.at(dem.ref - 60_000.0),
]

In [ ]:
layered_model_flat = lm.LayeredModel(
    list(more_itertools.interleave_longest(interfaces_flat, materials))
)
layered_model_topo = lm.LayeredModel(
    list(more_itertools.interleave_longest(interfaces_topo, materials))
)

In [ ]:
mr = sn.MeshResolution(
    reference_frequency=0.5, elements_per_wavelength=1.0, model_order=1
)

In [ ]:


mesh_flat = lm.mesh_from_domain(
    d,
    lm.MeshingProtocol(
        layered_model_flat,
        interlayer_coarsening_policy=[
            lm.InterlayerConstant(),
            # lm.InterlayerDoubling(),
            lm.InterlayerDoubling(),
            lm.InterlayerDoubling(),
            lm.InterlayerDoubling(),
        ],
        # ab=sn.AbsorbingBoundaryParameters(
        #     reference_velocity=4000.0,
        #     number_of_wavelengths=0.0,
        #     reference_frequency=1.0,
        # ),
    ),
    mr,
)

In [ ]:
mesh_topo = lm.mesh_from_domain(
    d,
    lm.MeshingProtocol(
        layered_model_topo,
        interlayer_coarsening_policy=[
            lm.InterlayerConstant(),
            # lm.InterlayerDoubling(),
            lm.InterlayerDoubling(),
            lm.InterlayerDoubling(),
            lm.InterlayerDoubling(),
        ],
        # ab=sn.AbsorbingBoundaryParameters(
        #     reference_velocity=4000.0,
        #     number_of_wavelengths=0.0,
        #     reference_frequency=1.0,
        # ),
    ),
    mr,
)

In [ ]:
# mesh_flat.write_h5("mesh_flat.h5")

In [ ]:
mesh_topo

In [ ]:
# Uncomment the following line to delete a potentially existing project for a fresh start
# !rm -rf $PROJECT_DIR
p = sn.Project.from_domain(path=PROJECT_DIR, domain=d, load_if_exists=True)

In [ ]:
#define sources and wavelets from excel file
import pandas as pd
import numpy as np
source_file=pd.read_csv('/srv/gamp/png/Paper/Aftershocks/new250_fault.csv')
d=source_file['rise'].astype(float)
f=d.to_list()
r=source_file['rupture'].astype(float)
rupt=r.to_list()


sources=[]
# wavelets=sn.simple_config.stf.Ricker(center_frequency=0.2)
wavelets=[]
for i in range(len(f)):
    wave_pt=sn.simple_config.stf.GaussianRate(half_duration_in_seconds=float(f[i])/2, time_shift_in_seconds=float(rupt[i]))
    wavelets.append(wave_pt)


for idx,row in source_file.iterrows():
    event_pt=sn.simple_config.source.cartesian.MomentTensorPoint3D(
    x=row.x,
    y=row.y,
    z=-row.z*1000,
    mxx= row.Mtt,
    myy= row.Mpp,
    mzz= row.Mrr,
    myz= -row.Mrp,
    # myz= row.Mrp,
    mxz= row.Mrt,
    mxy= -row.Mtp,
    # mxy= row.Mtp,
    # offset=-row.z*1000,
)


    sources.append(event_pt)

In [ ]:
# Create an array for the receivers
rec = sn.simple_config.receiver.cartesian.collections.SideSetArrayPoint3D(
    y=np.arange(9250000, 9410000, 10000),
    x=np.arange(520000, 810000, 10000),
    depth_in_meters=0.0,
    fields=["velocity"],
)
# If receivers are available
# rec=[]
# source_file=pd.read_csv('/srv/gamp/png/Paper/receiver.csv')
# for idx,row in source_file.iterrows():
#     recs=sn.simple_config.receiver.cartesian.SideSetPoint3D(
#         point=(row.X, row.Y , 0),
#         direction=(0, 0, 1),
#         side_set_name="z1",
#         fields=["velocity"],
#         station_code=row.Name,
#         # station_code=f"XX_{idx}",
#     )
#     rec.append(recs)

In [ ]:
# Adding the event to the project.
p += sn.Event(event_name="PNG", sources=sources, receivers=rec)

In [ ]:
abp=sc.boundary.Absorbing(side_sets=['x0','x1','y0','y1','z0'], taper_amplitude=0.8, width_in_meters=10000.0)

In [ ]:
# Event configuration , attenuation= False, boundary_conditions=abp

ec = sn.EventConfiguration(
    wavelet=wavelets,
    waveform_simulation_configuration=sn.WaveformSimulationConfiguration(
        end_time_in_seconds=200.0, attenuation= False, boundary_conditions=abp
    ),
)

In [ ]:
# # Create absorbing boundaries
# Note -- its not necessary here to use sponge layers really.

# ab = sn.AbsorbingBoundaryParameters(
#     reference_velocity=3000.0,
#     reference_frequency=0.5,
#     number_of_wavelengths=4.5,
# )

In [ ]:
# Prepare the simulation flat surface
# p += sn.SimulationConfiguration(
#     name="flat",
#     tensor_order=1,
#     model_configuration=mc,
#     event_configuration=ec,
#     absorbing_boundaries=ab,
#     elements_per_wavelength=1,
#     max_depth_in_meters=40000.0,
#     max_frequency_in_hertz=0.4,
#     topography_configuration=tc,
# )

p += sn.UnstructuredMeshSimulationConfiguration(
    name="flat", unstructured_mesh=mesh_flat, event_configuration=ec
)

In [ ]:
# Prepare simulation for tensor order 2 topography
# p += sn.SimulationConfiguration(
#     name="topo",
#     tensor_order=1,
#     model_configuration=mc,
#     event_configuration=ec,
#     absorbing_boundaries=ab,
#     elements_per_wavelength=1,
#     max_depth_in_meters=40000.0,
#     max_frequency_in_hertz=0.4,
#     topography_configuration=tc_topo,
# )

p += sn.UnstructuredMeshSimulationConfiguration(
    name="topo", unstructured_mesh=mesh_topo, event_configuration=ec
)

In [ ]:
# Visualize simulation configuration
p.viz.nb.simulation_setup("topo",["PNG"])

In [ ]:
# Launch simulation for tensor order 2
p.simulations.launch(
    "topo",
    events=["PNG"],
    site_name=SALVUS_FLOW_SITE_NAME,
    ranks_per_job=40,
    # wall_time_in_seconds_per_job=100000,
    extra_output_configuration={
        "surface_data": {
            "sampling_interval_in_time_steps": 1,
            "fields": ["velocity"],
            "side_sets": ["z1"],
        },
        "memory_per_rank_in_MB": 2000.0,
    },
)
p.simulations.query(block=True)

In [ ]:
# Launch simulation for tensor order 1 flat
p.simulations.launch(
    "flat",
    events=["PNG"],
    site_name=SALVUS_FLOW_SITE_NAME,
    ranks_per_job=40,
    # wall_time_in_seconds_per_job=100000,
    extra_output_configuration={
        "surface_data": {
            "sampling_interval_in_time_steps": 1,
            "fields": ["velocity"],
            "side_sets": ["z1"],
        },
        "memory_per_rank_in_MB": 2000.0,
    },
)
p.simulations.query(block=True)

In [ ]:
# Visualize time series
# p.simulations.query(block=True)
p.viz.nb.waveforms([ "flat","topo"], receiver_field="velocity")

In [ ]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.tri as tri

In [ ]:
f1=h5py.File("/home/gamp/srv/png/Paper/Aftershocks/old_fault/EVENTS/PNG/WAVEFORM_DATA/INTERNAL/82/88/25c5fe5dce64/surface_data_output.h5", "r")
f2=h5py.File("/home/gamp/srv/png/Paper/Aftershocks/old_fault/EVENTS/PNG/WAVEFORM_DATA/INTERNAL/a5/12/42febb286737/surface_data_output.h5", "r")

In [ ]:
coordinates = f2["coordinates_ELASTIC_surface"][:]
data = f2["surface"]["velocity"][:]

# Compute the magnitude - the third dimensions is the components.
magnitude = np.linalg.norm(data, axis=2)

# Peak ground displacement.
pgv = np.max(magnitude, axis=0)

In [ ]:
triang = tri.Triangulation(
    coordinates[:, :, 0].flatten(), coordinates[:, :, 1].flatten()
)
plt.tripcolor(triang, pgv.flatten(), shading="gouraud")
plt.gca().set_aspect(1.0)
plt.colorbar()
plt.show()